In [ ]:
import os
import pickle
from sdypy import EMA
import numpy as np
import matplotlib.pyplot as plt 
from EMA_functions import *
import pandas as pd
import matplotlib.animation as animation
import matplotlib.patches as patches

import importlib
from scipy.ndimage import generic_filter
from scipy.ndimage import maximum_filter
import timeit

In [ ]:
%matplotlib qt
file_name = "Full_web_ecc2_new_Floc7_v0"        # Second attempt, ecc2
EMA_structure = EMA_Structure(file_name)
video = EMA_structure.open_video()
reference_image = (0, 100)#(0, 100)
bit_depth = 16

fig, ax = EMA_structure.plot_still_frame(video, reference_image)

In [ ]:
mean_image = EMA_structure.mean_image
c_vec = ['r', 'b', 'g']
# mean_image = mean_image/np.max(mean_image)
roi_size = (9, 9)
scale = .3  # Adjust this value to scale the eigenvalues
origin = [0, 0]

pixel1 = (222, 485) # (y, x)
ref_img1 = mean_image[pixel1[0]-roi_size[0]//2:pixel1[0]+roi_size[0]//2+1, pixel1[1]-roi_size[1]//2:pixel1[1]+roi_size[1]//2+1]

Xi, Xj = np.meshgrid(np.arange(ref_img1.shape[1]), np.arange(ref_img1.shape[0]))
gy1, gx1 = np.gradient(ref_img1)
A1 = np.array([gx1.flatten(), gy1.flatten()]).T
ATA1 = A1.T @ A1
eig_vals1, eig_vecs1 = np.linalg.eig(ATA1)
eig_vec1_0 = eig_vecs1[:,0]
eig_vec1_1 = eig_vecs1[:,1]
scaled_eig_vec1_0 = np.sqrt(eig_vals1[0]) * eig_vec1_0 * scale
scaled_eig_vec1_1 = np.sqrt(eig_vals1[1]) * eig_vec1_1 * scale

pixel2 =  (204, 394)# (y, x)
ref_img2 = mean_image[pixel2[0]-roi_size[0]//2:pixel2[0]+roi_size[0]//2+1, pixel2[1]-roi_size[1]//2:pixel2[1]+roi_size[1]//2+1]
gy2, gx2 = np.gradient(ref_img2)
A2 = np.array([gx2.flatten(), gy2.flatten()]).T
ATA2 = A2.T @ A2
eig_vals2, eig_vecs2 = np.linalg.eig(ATA2)
eig_vec2_0 = eig_vecs2[:,0]
eig_vec2_1 = eig_vecs2[:,1]
scaled_eig_vec2_0 = np.sqrt(eig_vals2[0]) * eig_vec2_0 * scale
scaled_eig_vec2_1 = np.sqrt(eig_vals2[1]) * eig_vec2_1 * scale

pixel3 = (202, 386) # (y, x)
ref_img3 = mean_image[pixel3[0]-roi_size[0]//2:pixel3[0]+roi_size[0]//2+1, pixel3[1]-roi_size[1]//2:pixel3[1]+roi_size[1]//2+1]
gy3, gx3 = np.gradient(ref_img3)
A3 = np.array([gx3.flatten(), gy3.flatten()]).T
ATA3 = A3.T @ A3
eig_vals3, eig_vecs3 = np.linalg.eig(ATA3)
eig_vec3_0 = eig_vecs3[:,0]
eig_vec3_1 = eig_vecs3[:,1]
scaled_eig_vec3_0 = np.sqrt(eig_vals3[0]) * eig_vec3_0 * scale
scaled_eig_vec3_1 = np.sqrt(eig_vals3[1]) * eig_vec3_1 * scale

fig, ax = plt.subplots(figsize=(18, 8))
fig.tight_layout()
ax.set_xticks([])
ax.set_yticks([])
ax.scatter(pixel1[1], pixel1[0], marker='s', c=c_vec[0], s=50)
ax.scatter(pixel2[1], pixel2[0], marker='s', c=c_vec[1], s=50)
ax.scatter(pixel3[1], pixel3[0], marker='s', c=c_vec[2], s=50)
ax.imshow(mean_image, cmap='gray')

I_lim = 30000
pix_lim = roi_size[0] - 1
scale = 80000
scale2 = 200000

fig, ax = plt.subplots(3, 3)
fig.set_size_inches(10, 10)

ax[0, 0].set_aspect('equal')
ax[0, 1].set_aspect('equal')
ax[0, 2].set_aspect('equal')
ax[1, 0].set_aspect('equal')
ax[1, 1].set_aspect('equal')
ax[1, 2].set_aspect('equal')

ax[0,0].imshow(ref_img1, cmap='gray', vmin=0, vmax=2**bit_depth-1)
rect = patches.Rectangle((-0.5, -0.5), roi_size[0], roi_size[1], edgecolor=c_vec[0], facecolor='none')
lw = 10
rect.set_linewidth(lw)  # Set the line thickness to 2
ax[0,0].add_patch(rect)
ax[0,0].invert_yaxis()
ax[0,1].quiver(Xi, Xj, gx1, gy1, scale=scale2)
ax[0,2].scatter(gx1, gy1, s=1, c='k')
ax[0,2].quiver(*origin, *scaled_eig_vec1_0, color=['r'], scale=scale)
ax[0, 2].quiver(*origin, *scaled_eig_vec1_1, color=['b'], scale=scale)
ax[0, 1].set_xlim(0, pix_lim)
ax[0, 1].set_ylim(0, pix_lim)
ax[0, 2].set_xlim(-I_lim, I_lim)
ax[0, 2].set_ylim(-I_lim, I_lim)

ax[1,0].imshow(ref_img2, cmap='gray', vmin=0, vmax=2**bit_depth-1)
rect = patches.Rectangle((-0.5, -0.5), roi_size[0], roi_size[1], edgecolor=c_vec[1], facecolor='none')
rect.set_linewidth(lw)  # Set the line thickness to 2
ax[1,0].add_patch(rect)
ax[1,0].invert_yaxis()
ax[1,1].quiver(Xi, Xj, gx2, gy2, scale=scale2)
ax[1,2].scatter(gx2, gy2, s=1, c='k')
ax[1,2].quiver(*origin, *scaled_eig_vec2_0, color=['r'], scale=scale)
ax[1, 2].quiver(*origin, *scaled_eig_vec2_1, color=['b'], scale=scale)
ax[1, 1].set_xlim(0, pix_lim)
ax[1, 1].set_ylim(0, pix_lim)
ax[1, 2].set_xlim(-I_lim, I_lim)
ax[1, 2].set_ylim(-I_lim, I_lim)

ax[2,0].imshow(ref_img3, cmap='gray', vmin=0, vmax=2**bit_depth-1)
rect = patches.Rectangle((-0.5, -0.5), roi_size[0], roi_size[1], edgecolor=c_vec[2], facecolor='none')
rect.set_linewidth(lw)  # Set the line thickness to 2
ax[2,0].add_patch(rect)
ax[2,0].invert_yaxis()
ax[2,1].quiver(Xi, Xj, gx3, gy3, scale=scale2)
ax[2,2].scatter(gx3, gy3, s=1, c='k')
ax[2,2].quiver(*origin, *scaled_eig_vec3_0, color=['r'], scale=scale)
ax[2, 2].quiver(*origin, *scaled_eig_vec3_1, color=['b'], scale=scale)
ax[2, 1].set_xlim(0, pix_lim)
ax[2, 1].set_ylim(0, pix_lim)
ax[2, 2].set_xlim(-I_lim, I_lim)
ax[2, 2].set_ylim(-I_lim, I_lim)

eig_vals1, eig_vals2, eig_vals3

In [ ]:
roi_size = (9, 9)
def eig_filter_small(pixel_list):
    ref_img = pixel_list.reshape(roi_size)
    gy, gx = np.gradient(ref_img)
    A = np.array([gx.flatten(), gy.flatten()]).T
    ATA = A.T @ A
    eig_vals = np.linalg.eigvals(ATA)
    return min(eig_vals)

def eig_filter_direct(pixel_list):
    ref_img = pixel_list.reshape(roi_size)
    gy, gx = np.gradient(ref_img)
    A = np.stack((gx, gy), axis=-1)
    ATA = np.tensordot(A, A, axes=([0, 1], [0, 1]))
    # A = np.array([gx.flatten(), gy.flatten()]).T
    # ATA = A.T @ A
    m = (ATA[0, 0] + ATA[1, 1])/2
    p = ATA[0, 0] * ATA[1, 1] - ATA[0, 1] * ATA[1, 0]
    return m - np.sqrt(m**2 - p)


eig0 = eig_filter_small(ref_img2.flatten())
eig1 = eig_filter_small(ref_img3.flatten())

eig0_direct = eig_filter_direct(ref_img2.flatten())
eig1_direct = eig_filter_direct(ref_img3.flatten())


(eig0, eig1), (eig0_direct, eig1_direct)



In [ ]:
# # Define the setup code
# setup_code = '''
# import numpy as np
# from scipy.ndimage import generic_filter

# roi_size = (9, 9)
# mean_image = np.random.rand(512, 1024)

# def eig_filter_small(pixel_list):
#     ref_img = pixel_list.reshape(roi_size)
#     gy, gx = np.gradient(ref_img)
#     A = np.array([gx.flatten(), gy.flatten()]).T
#     ATA = A.T @ A
#     eig_vals = np.linalg.eigvals(ATA)
#     return min(eig_vals)

# def eig_filter_direct(pixel_list):
#     ref_img = pixel_list.reshape(roi_size)
#     gy, gx = np.gradient(ref_img)
#     A = np.array([gx.flatten(), gy.flatten()]).T
#     ATA = A.T @ A
#     m = np.trace(ATA)/2
#     p = np.linalg.det(ATA)
#     return m - np.sqrt(m**2 - p)

# def eig_filter_direct2(pixel_list):
#     ref_img = pixel_list.reshape(roi_size)
#     gy, gx = np.gradient(ref_img)
#     A = np.stack((gx, gy), axis=-1)
#     ATA = np.tensordot(A, A, axes=([0, 1], [0, 1]))
#     # A = np.array([gx.flatten(), gy.flatten()]).T
#     # ATA = A.T @ A
#     m = (ATA[0, 0] + ATA[1, 1])/2
#     p = ATA[0, 0] * ATA[1, 1] - ATA[0, 1] * ATA[1, 0]
#     return m - np.sqrt(m**2 - p)

# def eig_filter_complex(pixel_list):
#     pixel_list = pixel_list.reshape(-1, 2)
#     ATA = pixel_list.T @ pixel_list
#     m = (ATA[0, 0] + ATA[1, 1])/2
#     p = ATA[0, 0] * ATA[1, 1] - ATA[0, 1] * ATA[1, 0]
#     return m - np.sqrt(m**2 - p)
# '''

# # Define the code to measure execution time
# code_to_measure_small = '''
# mean_eig_img = generic_filter(mean_image, eig_filter_small, size=(9, 9))
# '''
# code_to_measure_direct = '''
# mean_eig_img_direct = generic_filter(mean_image, eig_filter_direct, size=(9, 9))
# '''
# code_to_measure_direct2 = '''
# mean_eig_img_direct2 = generic_filter(mean_image, eig_filter_direct2, size=(9, 9))
# '''
# code_to_measure_complex = '''
# gy, gx = np.gradient(mean_image)
# combined_gy_gx = np.stack((gy, gx), axis=-1)
# mean_eig_img_complex = generic_filter(combined_gy_gx, eig_filter_complex, size=(9, 9, 2))
# '''

# # Measure the execution time
# # small_time = timeit.timeit(code_to_measure_small, setup=setup_code, number=10)
# # direct_time = timeit.timeit(code_to_measure_direct, setup=setup_code, number=10)
# direct_time2 = timeit.timeit(code_to_measure_direct2, setup=setup_code, number=1)
# complex_time = timeit.timeit(code_to_measure_complex, setup=setup_code, number=1)

# small_time, direct_time, direct_time2, complex_time

In [ ]:
roi_size = (9, 9)
mean_image /= np.max(mean_image)
gy, gx = np.gradient(mean_image)
combined_gy_gx = np.stack((gy, gx), axis=-1)
def eig_filter_complex(pixel_list):
    pixel_list = pixel_list.reshape(-1, 2)
    ATA = pixel_list.T @ pixel_list
    # ATA = np.tensordot(A, A, axes=([0, 1], [0, 1]))
    m = (ATA[0, 0] + ATA[1, 1])/2
    p = ATA[0, 0] * ATA[1, 1] - ATA[0, 1] * ATA[1, 0]
    return m - np.sqrt(m**2 - p)

ref_img_y1 = gy[pixel1[0]-roi_size[0]//2:pixel1[0]+roi_size[0]//2+1, pixel1[1]-roi_size[1]//2:pixel1[1]+roi_size[1]//2+1]
ref_img_x1 = gx[pixel1[0]-roi_size[0]//2:pixel1[0]+roi_size[0]//2+1, pixel1[1]-roi_size[1]//2:pixel1[1]+roi_size[1]//2+1]
ref_img1   =  np.stack((ref_img_y1, ref_img_x1), axis=-1)
ref_img_y2 = gy[pixel2[0]-roi_size[0]//2:pixel2[0]+roi_size[0]//2+1, pixel2[1]-roi_size[1]//2:pixel2[1]+roi_size[1]//2+1]
ref_img_x2 = gx[pixel2[0]-roi_size[0]//2:pixel2[0]+roi_size[0]//2+1, pixel2[1]-roi_size[1]//2:pixel2[1]+roi_size[1]//2+1]
ref_img2   =  np.stack((ref_img_y2, ref_img_x2), axis=-1)
ref_img_y3 = gy[pixel3[0]-roi_size[0]//2:pixel3[0]+roi_size[0]//2+1, pixel3[1]-roi_size[1]//2:pixel3[1]+roi_size[1]//2+1]
ref_img_x3 = gx[pixel3[0]-roi_size[0]//2:pixel3[0]+roi_size[0]//2+1, pixel3[1]-roi_size[1]//2:pixel3[1]+roi_size[1]//2+1]
ref_img3   =  np.stack((ref_img_y3, ref_img_x3), axis=-1)
eig0 = eig_filter_complex(ref_img1.reshape(roi_size[0]**2, 2))
eig1 = eig_filter_complex(ref_img2.reshape(roi_size[0]**2, 2))
eig2 = eig_filter_complex(ref_img3.reshape(roi_size[0]**2, 2))
eig0, eig1, eig2

In [ ]:
fig, ax = plt.subplots(figsize=(12, 6))
ax.imshow(gx, cmap='gray')
ax.set_title('gx')
ax.set_xticks([])
ax.set_yticks([])
fig.show()

fig, ax = plt.subplots(figsize=(12, 6))
ax.imshow(gy, cmap='gray')
ax.set_title('gy')
ax.set_xticks([])
ax.set_yticks([])
fig.show()


In [ ]:
from skimage import data, transform
n = 10
scaler = 2
h = combined_gy_gx.shape[0]
w = combined_gy_gx.shape[1]
roi_size = (n, n, 2)
mean_eig_img = generic_filter(combined_gy_gx, eig_filter_complex, size=roi_size)
gy_ds = transform.downscale_local_mean(combined_gy_gx[:,:,0], (scaler, scaler))
gx_ds = transform.downscale_local_mean(combined_gy_gx[:,:,1], (scaler, scaler))
combined_gy_gx_ds   = np.stack((gy_ds, gx_ds), axis=-1)
mean_eig_img_ds     = generic_filter(combined_gy_gx_ds, eig_filter_complex, size=roi_size//scaler)

fig, ax = plt.subplots(2, figsize=(12, 6))
fig.tight_layout()
ax[0].imshow(mean_eig_img[:,:,1], cmap='gray')
ax[0].set_xticks([])
ax[0].set_yticks([])
ax[1].imshow(mean_eig_img_ds[:,:,1], cmap='gray')
ax[1].set_xticks([])
ax[1].set_yticks([])


In [ ]:
fig, ax = plt.subplots(figsize=(12, 6))
fig.tight_layout()
ax.set_xticks([])
ax.set_yticks([])
struct_image = np.copy(mean_image)
background_pixels = np.where(struct_image > 0.25)
struct_image[background_pixels] = 0
struct_eig_image = np.copy(mean_eig_img[:,:,1])
struct_eig_image[background_pixels] = 0
ax.imshow(struct_eig_image, cmap='gray')

In [ ]:
local_max = maximum_filter(struct_eig_image, size=4)
fig, ax = plt.subplots(figsize=(12, 6))
fig.tight_layout()
ax.set_xticks([])
ax.set_yticks([])
ax.imshow(mean_image, cmap='gray')

local_maxima = (struct_eig_image == local_max)
local_maxima[background_pixels] = False
local_maxima[local_max < 0.4] = False
y_coords, x_coords = np.where(local_maxima)
ax.scatter(x_coords, y_coords, c='red', s=10)
fig.show()




In [ ]:
roi_size_vec = [3, 5, 7, 9, 11, 13, 15] #[9] #
fig, ax = plt.subplots(3, 3)
for i, roi_size_n in enumerate(roi_size_vec):
    roi_size = (roi_size_n, roi_size_n, 2)
    mean_eig_img = generic_filter(combined_gy_gx, eig_filter_complex, size=roi_size)
    row = i // 3
    col = i % 3
    ax[row, col].imshow(mean_eig_img[:,:,1], cmap='gray')
    ax[row, col].set_title('ROI size: ' + str(roi_size))


In [ ]:
ax[row, col].imshow(mean_eig_img[:,:,1], cmap='gray')
ax[row, col].set_title('ROI size: ' + str(roi_size))